In [1]:
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

def load_csv(filename):
    dataset = list()
    with open(filename, 'r', encoding='utf-8-sig') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

def to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

def to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

def evaluate(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        print("Predicted", predicted)
        actual = [row[-1] for row in fold]
        print("Actual:",actual, "\n")
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

def sigmoid(activation):
    return 1.0 / (1.0 + exp(-activation))

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = sigmoid(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

def activate(output):
    return output * (1.0 - output)

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * activate(neuron['output'])

def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)

def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
#     print("Network", network, "\n")
    return(predictions)

In [2]:
# load and prepare training data
filename = 'E:\\train.csv'
train = load_csv(filename)
for i in range(len(train[0])-1):
    to_float(train, i)
# convert class column to integers
to_int(train, len(train[0])-1)
# normalize input variables
minmax = dataset_minmax(train)
normalize_dataset(train, minmax)

# load and prepare testing data
filename = 'E:\\test.csv'
test = load_csv(filename)
for i in range(len(test[0])-1):
    to_float(test, i)
# convert class column to integers
to_int(test, len(test[0])-1)
# normalize input variables
minmax = dataset_minmax(test)
normalize_dataset(test, minmax)

res = back_propagation(train, test, 0.2, 5000,9)
target = [row[-1] for row in test]
print(target)
print(res)
accuracy_metric(target,res)


[0, 0, 0, 1, 1, 1, 2, 2, 2]
[0, 2, 0, 1, 0, 1, 2, 2, 2]


77.77777777777779

In [3]:
# Test with 10 folds
seed(1)
# load and prepare data
filename = 'E:\\dataset.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
    to_float(dataset, i)
# convert class column to integers
to_int(dataset, len(dataset[0])-1)
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# print("minmax", minmax)
# print("Normalized data:", dataset)

In [4]:
# evaluate algorithm
n_folds = 6
l_rate = 0.2
n_epoch = 20000
n_hidden = 9
scores = evaluate(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Predicted [1, 0, 0, 1, 0, 0, 1, 1, 0, 0]
Actual: [1, 2, 0, 0, 0, 1, 2, 1, 2, 0] 

Predicted [2, 2, 0, 1, 2, 1, 0, 1, 2, 0]
Actual: [2, 2, 0, 1, 2, 1, 0, 1, 2, 0] 

Predicted [0, 0, 2, 1, 1, 1, 2, 0, 1, 0]
Actual: [0, 1, 0, 1, 1, 2, 2, 1, 1, 1] 

Predicted [2, 2, 2, 1, 0, 1, 2, 2, 0, 1]
Actual: [0, 2, 1, 1, 0, 2, 2, 2, 0, 1] 

Predicted [0, 2, 0, 1, 0, 1, 1, 2, 2, 1]
Actual: [0, 0, 0, 1, 0, 2, 0, 2, 2, 1] 

Predicted [2, 0, 2, 1, 0, 1, 2, 0, 0, 1]
Actual: [2, 0, 2, 1, 0, 1, 2, 0, 2, 1] 

Scores: [50.0, 100.0, 50.0, 70.0, 70.0, 90.0]
Mean Accuracy: 71.667%
